<a href="https://colab.research.google.com/github/M-110/testing-with-pytest/blob/main/06_Configuration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Config files

* pytest.ini: Main file which allows you to change the default behavior.

* conftest.py: local plugin to allow hook functions and fixtures for the directory where the confest.py file exists and all subdirectories. 

* \_\_init__.py: when put into every subdirectory, this file allows you to have the identical test filenames in multiple test directories.

* setup.py: This is a file in the ini format that effects the behavior of setup.py.

In [ ]:
%%writefile pytest.ini
[pytest]
addopts = '-rsxX -l --tb=short --strict'
xfail_strict = true
...

In [ ]:
%%writefile setup.cfg
# Package specific stuff

[tool:pytest]
addopts = '-rsxX -l --tb=short --strict'
xfail_strict = true
...

In [ ]:
!pytest --help 

# Registering markers to avoid marker typos

In [ ]:
%%writefile pytest.ini
[pytest]
markers =
  smoke: Run the smoke tests
  get: Run the test functions that test get

Writing pytest.ini


In [ ]:
!pytest --markers

@pytest.mark.smoke: Run the smoke tests

@pytest.mark.get: Run the test functions that test get

@pytest.mark.skip(reason=None): skip the given test function with an optional reason. Example: skip(reason="no way of currently testing this") skips the test.

@pytest.mark.skipif(condition): skip the given test function if eval(condition) results in a True value.  Evaluation happens within the module global context. Example: skipif('sys.platform == "win32"') skips the test if we are on the win32 platform. see http://pytest.org/latest/skipping.html

@pytest.mark.xfail(condition, reason=None, run=True, raises=None, strict=False): mark the test function as an expected failure if eval(condition) has a True value. Optionally specify a reason for better reporting and run=False if you don't even want to execute the test function. If only specific exception(s) are expected, you can list them in raises, and if the test fails in other ways, it will be reported as a true failure. See http://pytest.or

In [ ]:
%%writefile test_stuff.py
import pytest

@pytest.mark.smoke()
def test_this():
  assert 1 == 1

@pytest.mark.smok()
def test_that():
  assert True

Writing test_stuff.py


In [ ]:
!pytest --strict -v --tb=short

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile: pytest.ini
collected 0 items / 1 errors                                                   

==================================== ERRORS ====================================
________________________ ERROR collecting test_stuff.py ________________________
test_stuff.py:7: in <module>
    @pytest.mark.smok()
/usr/local/lib/python2.7/dist-packages/_pytest/mark/structures.py:367: in __getattr__
    self._check(name)
/usr/local/lib/python2.7/dist-packages/_pytest/mark/structures.py:383: in _check
    raise AttributeError("%r not a registered marker" % (name,))
E   AttributeError: 'smok' not a registered marker
!!!!!!!!!!!!!!!!!!! Interrupted: 1 errors during collection !!!!!!!!!!!!!!!!!!!!
=========================== 1 error in 0.06 seconds ============================


# Require minimum pytest version

In [ ]:
%%writefile pytest.ini
[pytest]
minversion = 3.0

Overwriting pytest.ini


In [ ]:
!pytest

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile: pytest.ini
collected 2 items                                                              

test_stuff.py ..                                                         [100%]

=========================== 2 passed in 0.01 seconds ===========================


# stopping pytest from looking in the wrong places.

In [ ]:
%%writefile pytest.ini
[pytest]
norecursedirs = venv

Overwriting pytest.ini


In [ ]:
!mkdir 'venv'

mkdir: cannot create directory ‘venv’: File exists


In [ ]:
%%writefile venv/test_other.py
def test_hidden():
  assert False

Overwriting venv/test_other.py


In [ ]:
!pytest -v # Didn't search in the venv directory for a test

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile: pytest.ini
collected 2 items                                                              

test_stuff.py::test_this PASSED                                          [ 50%]
test_stuff.py::test_that PASSED                                          [100%]

=========================== 2 passed in 0.01 seconds ===========================


# Specifying Test Directory Locations

testspaths

In [ ]:
%%writefile pytest.ini
[pytest]
testpaths = tests

Overwriting pytest.ini


In [ ]:
!mkdir tests

In [ ]:
%%writefile tests/test_more.py
def test_more():
  assert 1


Writing tests/test_more.py


In [ ]:
!pytest

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile: pytest.ini
collected 1 item                                                               

tests/test_more.py .                                                     [100%]

=========================== 1 passed in 0.02 seconds ===========================


# Changing test discovery rules

by default it looks for test_* or *_test and classes use Test* *Test

You can change this to find things with *Suite:

In [ ]:
%%writefile pytest.ini
[pytest]
python_classes = *Test Test* *Suite

Overwriting pytest.ini


In [ ]:
%%writefile test_suite.py
class GreatSuite:
  def test_this(self):
    assert 1
  
  def test_that(self):
    assert 1

Overwriting test_suite.py


In [ ]:
!pytest -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile: pytest.ini
collected 5 items                                                              

test_stuff.py::test_this PASSED                                          [ 20%]
test_stuff.py::test_that PASSED                                          [ 40%]
test_suite.py::GreatSuite::test_this PASSED                              [ 60%]
test_suite.py::GreatSuite::test_that PASSED                              [ 80%]
tests/test_more.py::test_more PASSED                                     [100%]

=========================== 5 passed in 0.02 seconds ===========================


Finding tests that start with check_

In [ ]:
%%writefile pytest.ini
[pytest]
python_classes = *Test Test* *Suite
python_files = test_* *_test check_*
python_functions = test_* check_*

Overwriting pytest.ini


In [ ]:
%%writefile check_stuff.py
def check_this():
  assert 1

def check_that():
  assert 2

Writing check_stuff.py


In [ ]:
!pytest -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile: pytest.ini
collected 7 items                                                              

check_stuff.py::check_this PASSED                                        [ 14%]
check_stuff.py::check_that PASSED                                        [ 28%]
test_stuff.py::test_this PASSED                                          [ 42%]
test_stuff.py::test_that PASSED                                          [ 57%]
test_suite.py::GreatSuite::test_this PASSED                              [ 71%]
test_suite.py::GreatSuite::test_that PASSED                              [ 85%]
tests/test_more.py::test_more PASSED                                     [100%]

=========================== 7 passed in 0.03 seconds ===========================


# Disallowing XPASS

seems like this should always be set

In [ ]:
%%writefile xpass_test.py
import pytest

@pytest.mark.xfail
def test_xpass():
  pass

Writing xpass_test.py


In [ ]:
!pytest xpass_test.py -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile: pytest.ini
collected 1 item                                                               

xpass_test.py::test_xpass XPASS                                          [100%]

========================== 1 xpassed in 0.01 seconds ===========================


In [ ]:
%%writefile pytest.ini
[pytest]
xfail_strict = true

Overwriting pytest.ini


In [ ]:
!pytest xpass_test.py -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile: pytest.ini
collected 1 item                                                               

xpass_test.py::test_xpass FAILED                                         [100%]

=================================== FAILURES ===================================
__________________________________ test_xpass __________________________________
[XPASS(strict)] 
=========================== 1 failed in 0.01 seconds ===========================


#Avoiding Filename Collisions

by adding an __ini__.py file into the directories

In [ ]:
!mkdir a b

In [ ]:
%%writefile a/test_me.py
def test_it():
  assert 1

Writing a/test_me.py


In [ ]:
%%writefile b/test_me.py
def test_it():
  assert 1

Writing b/test_me.py


In [ ]:
!pytest

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile: pytest.ini
collected 5 items / 1 errors                                                   

==================================== ERRORS ====================================
________________________ ERROR collecting b/test_me.py _________________________
import file mismatch:
imported module 'test_me' has this __file__ attribute:
  /content/a/test_me.py
which is not the same as the test file we want to collect:
  /content/b/test_me.py
HINT: remove __pycache__ / .pyc files and/or use a unique basename for your test file modules
!!!!!!!!!!!!!!!!!!! Interrupted: 1 errors during collection !!!!!!!!!!!!!!!!!!!!
=========================== 1 error in 0.08 seconds ============================


In [ ]:
!touch a/__init__.py b/__init__.py

In [ ]:
!pytest -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile: pytest.ini
collected 6 items                                                              

test_stuff.py::test_this PASSED                                          [ 16%]
test_stuff.py::test_that PASSED                                          [ 33%]
xpass_test.py::test_xpass FAILED                                         [ 50%]
a/test_me.py::test_it PASSED                                             [ 66%]
b/test_me.py::test_it PASSED                                             [ 83%]
tests/test_more.py::test_more PASSED                                     [100%]

=================================== FAILURES ===================================
__________________________________ test_xpass __________________________________
[XPASS(strict)] 
====================== 1